<a href="https://colab.research.google.com/github/Yuri-Cordeiro/FarmTechSolutions/blob/main/CulturaAgricola.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistema de Gestão Agrícola

Este notebook implementa um sistema de gestão agrícola para calcular áreas de plantio e manejo de insumos para diferentes culturas. Atualmente, ele suporta as culturas de **Soja, Café, Milho, Feijão** e **Cana-de-Açúcar**.


## Importações e Definições Auxiliares

Nesta seção, importamos as bibliotecas necessárias e definimos funções auxiliares para o sistema, como a limpeza do terminal e a entrada de valores numéricos.


In [14]:
import csv
from IPython.display import clear_output

import rpy2.robjects as ro #importados para conseguir usar o python com o R
import pandas as pd

# Importações específicas da biblioteca rpy2
from rpy2.robjects import r
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr

# Carregando a extensão rpy2.ipython (necessário para Jupyter)
%load_ext rpy2.ipython

def input_numerico(texto_input) -> int:
    """
    Solicita um input numérico do usuário e valida se é um número inteiro.

    :param texto_input: Texto a ser exibido para o usuário
    :return: Valor numérico inteiro
    """
    while True:
        valor_input = input(texto_input)
        if valor_input.isdigit() and int(valor_input) > 0:
            return int(valor_input)
        print(">> Valor inválido! Digite um número inteiro positivo.")


def mostra_menu(titulo, opcoes):
    """
    Exibe um menu com opções para o usuário.

    :param titulo: Título do menu
    :param opcoes: Lista de opções do menu
    """
    max_length = max(len(option) for option in opcoes)
    max_length = len(titulo) if len(titulo) >= max_length else max_length
    border_length = max_length + 6
    print("\n╔" + "═" * border_length + "╗")
    print(f"║ {titulo.center(border_length - 2)} ║")
    print("╚" + "═" * border_length + "╝")
    for i, option in enumerate(opcoes, start=1):
        print(f"║ {i} - {option.ljust(max_length)} ║")
    print("╚" + "═" * border_length + "╝")
    print("║ S - Sair ".ljust(max_length + 7) + "║")
    print("╚" + "═" * border_length + "╝")


def seleciona_escolha(titulo, opcoes):
    """
    Garante que o usuario escolha entre o vetor de opções informado.

    :param titulo: Título do menu
    :param opcoes: Lista de opções do menu
    """
    while True:
        mostra_menu(titulo, opcoes)
        choice = input("\n>> ").strip().upper()
        if choice == "S":
            return "S"
        try:
            choice = int(choice)
            if 1 <= choice <= len(opcoes):
                return choice
            else:
                print(">> Opção inválida!")
        except ValueError:
            print(">> Opção inválida!")



The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


## Definição da Classe CulturaAgricola

A classe `CulturaAgricola` é responsável por armazenar informações sobre uma cultura agrícola específica, incluindo as ruas da plantação, a forma geométrica usada para calcular a área de plantio e a quantidade de insumo por metro. Também inclui métodos para calcular a área de plantio e o manejo de insumos.


In [15]:
class CulturaAgricola:
    def __init__(self, nome, ruas, produtos, formas_geometricas):
        """
        Inicializa a cultura agrícola.

        :param nome: Nome da cultura (e.g., "Café", "Cana-de-Açúcar")
        :param ruas: Lista com os comprimentos das ruas em metros
        :param produtos: Dicionário para armazenar produtos e suas respectivas doses
        :param formas_geometricas: Lista de formas geométricas suportadas
        """
        self.nome = nome
        self.ruas = ruas
        self.produtos = produtos
        self.formas_geometricas = formas_geometricas
        self.areas_calculadas = []

    def add_rua(self, rua):
        """
        Atualiza vetor das ruas
        """
        self.ruas.append(rua)

    def remover_rua(self):
        """
        Remove item do vetor de ruas.
        """
        if len(self.ruas) > 0:
            indice = seleciona_escolha(">> Remover rua", [str(r) for r in self.ruas])

            if indice != "S":
                del self.ruas[indice - 1]

        else:
            print(">> Vetor de ruas já esta vazio")

    def limpa_ruas(self):
        """
        Limpa o vetor de ruas
        :return:
        """
        self.ruas = []

    def limpa_areas_calculadas(self):
        """
        Limpa o historico de areas calculadas
        :return:
        """
        self.areas_calculadas = []

    def calcular_area_retangulo(self):
        """
        Calcula a área de plantio na forma retangular. Altura X Largura
        :return: Área calculada
        """
        comprimento = input_numerico("\n>> Digite o valor do comprimento: ")
        largura = input_numerico("\n>> Digite o valor da largura: ")
        calculo_area = comprimento * largura

        self.areas_calculadas.append(f"Forma: Retângulo | Comprimento: {comprimento}m | Largura: {largura}m | "
                                     f"Área: {calculo_area}m²")

        print(f">> Cálculo: {comprimento} * {largura}")
        print(f">> Área do plantio: {calculo_area}m².")
        return calculo_area

    def calcular_area_triangulo(self):
        """
        Calcula a área de plantio na forma triangular. (Base X Altura) / 2
        :return: Área calculada
        """
        base = input_numerico("\n>> Digite o valor da base: ")
        altura = input_numerico("\n>> Digite o valor da altura: ")
        calculo_area = (base * altura) / 2

        self.areas_calculadas.append(f"Forma: Triângulo | Base:{base}m | Altura:{altura}m | Área:{calculo_area}m²")

        print(f">> Cálculo: {base} * {altura} / 2")
        print(f">> Área do plantio: {calculo_area}m².")
        return calculo_area

    def calcular_area_trapezio(self):
        """
        Calcula a área de plantio na forma de trapézio. ((base_maior + base_menor) * altura) / 2
        :return: Área calculada
        """
        base_maior = input_numerico("\n>> Digite o valor da base maior: ")
        base_menor = input_numerico("\n>> Digite o valor da base menor: ")
        altura = input_numerico("\n>> Digite o valor da altura: ")
        calculo_area = ((base_maior + base_menor) * altura) / 2

        self.areas_calculadas.append(f"Forma: Trapézio | Base maior: {base_maior}m | "
                                     f"Base menor: {base_menor}m | Altura: {altura}m | Área: {calculo_area}m²")

        print(f">> Cálculo: ({base_maior} + {base_menor}) * {altura} / 2")
        print(f">> Área do plantio: {calculo_area}m².")

        return calculo_area

    def calcular_area_plantio(self, forma_geometrica):
        """
        Calcula a área de plantio com base na forma geométrica escolhida.

        :param forma_geometrica: String indicando a forma geométrica ("retangulo", "triangulo", "trapezio")
        :return: Área calculada
        """
        if forma_geometrica == "retangulo":
            return self.calcular_area_retangulo()
        elif forma_geometrica == "triangulo":
            return self.calcular_area_triangulo()
        elif forma_geometrica == "trapezio":
            return self.calcular_area_trapezio()

    def adicionar_produto(self, nome_produto, insumo_por_metro):
        """
        Adiciona um produto ao cálculo de manejo de insumos.

        :param nome_produto: Nome do produto (e.g., "Fosfato", "Herbicida")
        :param insumo_por_metro: Quantidade de insumo a ser aplicada por metro em mL
        """
        self.produtos[nome_produto] = insumo_por_metro

    def remover_produto(self, nome_produto):
        """
        Remove um produto da cultura agrícola.

        :param nome_produto: Nome do produto a ser removido
        """
        if nome_produto in self.produtos:
            del self.produtos[nome_produto]
        else:
            print(f">> Produto {nome_produto} não encontrado.")

    def calcular_manejo_insumos(self):
        """
        Calcula a quantidade total de insumos necessária para cada produto.

        :return: Dicionário com a quantidade total de cada insumo em litros
        """
        total_insumos = {}
        for produto, insumo_por_metro in self.produtos.items():
            total_insumo = sum(rua * (insumo_por_metro / 1000) for rua in self.ruas)
            total_insumos[produto] = total_insumo
        return total_insumos

    def exibir_areas_calculadas(self):
        """
        Exibe historico das áreas calculadas.
        """
        [print(f">> {h}") for h in self.areas_calculadas]

    def exibir_informacoes(self):
        """
        Exibe as informações da cultura agrícola.
        """
        print(f">> Cultura: {self.nome}")
        print(f">> Comprimento das ruas em metro: {self.ruas}")
        print(">> Produtos e insumo por metro (mL):")
        for produto, insumo in self.produtos.items():
            print(f"   - {produto}: {insumo} mL")

    def gerar_csv(self, nome_arquivo='dados_agricolas.csv'):
        with open(nome_arquivo, mode='w', newline='') as file:
            writer = csv.writer(file)

            colunas = ['Cultura', 'Numero de Ruas', 'Produto', 'Quantidade Aplicada por Metro', 'Total de Insumos Necessarios']
            writer.writerow(colunas)

            for produto, insumo_por_metro in self.produtos.items():
                total_insumos = sum(rua * (insumo_por_metro / 1000) for rua in self.ruas)
                writer.writerow([self.nome, self.ruas, produto, insumo_por_metro, total_insumos])

        print(f"CSV '{nome_arquivo}' gerado com sucesso.")

## Definição de Valores Iniciais

Nesta seção, definimos os valores iniciais necessários para a execução do sistema. Esses valores incluem a criação das instâncias das culturas agrícolas, o mapeamento de produtos aos seus respectivos insumos por metro e a configuração das ruas para cada cultura.

**Importante:** Os valores definidos aqui são apenas valores iniciais e são **opcionais**. Você pode alterar esses valores ao longo do uso do sistema, conforme necessário. Isso permite ajustar as configurações para refletir diferentes cenários ou requisitos específicos da sua análise agrícola.

Esses valores são usados para realizar os cálculos de área de plantio e manejo de insumos. Se você quiser modificar qualquer configuração, basta alterar os valores na seção apropriada do código.


### Nota Importante

O cálculo de área plantada assume que as medidas fornecidas estão em metros. Certifique-se de converter outras unidades antes de inseri-las.


In [16]:
# Valores iniciais das ruas
ruas = {
    'soja': [150, 200, 250, 180, 220],
    'cafe': [80, 90, 100, 85, 95],
    'milho': [120, 150, 180, 130, 160],
    'feijao': [100, 120, 110, 105, 115],
    'cana': [200, 250, 300, 220, 270],
}

# Formas geométricas aceitas
formas_geometricas = {
    'soja': ['retangulo', 'triangulo'],
    'cafe': ['retangulo'],
    'milho': ['retangulo', 'trapezio'],
    'feijao': ['triangulo', 'trapezio'],
    'cana': ['triangulo', 'trapezio'],
}

# Produtos utilizados
produtos = {
    'soja': {'Herbicida': 50, 'Fertilizante': 30, 'Inoculante': 20},
    'cafe': {'Fosfato': 40, 'Adubo': 30, 'Calda Bordalesa': 60},
    'milho': {'Herbicida': 45, 'Fertilizante': 25, 'Inseticida': 15},
    'feijao': {'Fertilizante': 20, 'Herbicida': 40, 'Inoculante': 10},
    'cana': {'Adubo': 50, 'Herbicida': 70, 'Fertilizante': 30},
}

# Vetor da classe de Culturas agricolas
culturas = [
    CulturaAgricola(nome="Soja", ruas=ruas['soja'], produtos=produtos['soja'],
                    formas_geometricas=formas_geometricas['soja']),
    CulturaAgricola(nome="Café", ruas=ruas['cafe'], produtos=produtos['cafe'],
                    formas_geometricas=formas_geometricas['cafe']),
    CulturaAgricola(nome="Milho", ruas=ruas['milho'], produtos=produtos['milho'],
                    formas_geometricas=formas_geometricas['milho']),
    CulturaAgricola(nome="Feijão", ruas=ruas['feijao'], produtos=produtos['feijao'],
                    formas_geometricas=formas_geometricas['feijao']),
    CulturaAgricola(nome="Cana-de-Açúcar", ruas=ruas['cana'], produtos=produtos['cana'],
                    formas_geometricas=formas_geometricas['cana'])
]

## Como Usar Este Notebook

1. **Selecione a cultura**: Escolha a cultura agrícola para a qual você deseja calcular a área de plantio ou o manejo de insumos.
2. **Adicione ruas**: Insira o comprimento das ruas para a cultura selecionada.
3. **Defina os produtos**: Defina o produto e a quantidade de insumo necessária por metro de rua.
4. **Calcule a área ou o manejo de insumos**: Escolha a operação que deseja realizar e o sistema fornecerá os resultados.


In [17]:
def main():
    cultura_atual = None
    acao_rua = False
    acao_insumo = False
    acao_calculo = False
    acao_dados_meterologicos = False
    sair = False

    while not sair:
        # Selecionar a cultura
        if not cultura_atual:
            clear_output(wait=True)

            opcoes = [c.nome for c in culturas]
            indice = seleciona_escolha("Culturas", opcoes)

            if indice == "S":
                sair = True

            else:
                cultura_atual = culturas[indice - 1]

        if cultura_atual:
            clear_output(wait=True)

            if not acao_rua and not acao_insumo and not acao_calculo:
                opcoes = ["Ruas", "Insumos", "Calculo", "Exibir informações", "Exportar Dados", "Consultar Dados Meteorológicos", "Voltar"]
                input_acao = seleciona_escolha(f"{cultura_atual.nome} > Ações", opcoes)

                if input_acao == 1:
                    acao_rua = True

                if input_acao == 2:
                    acao_insumo = True

                if input_acao == 3:
                    acao_calculo = True

                if input_acao == 4:
                    cultura_atual.exibir_informacoes()
                    input(">> Pressione uma tecla para continuar...")

                if input_acao == 5:
                    nome_arquivo = input(">> Digite o nome de saida do arquivo: ")
                    cultura_atual.gerar_csv(nome_arquivo)
                                    
                if input_acao == 6:
                    acao_dados_meterologicos = True

                if input_acao == 7:
                    cultura_atual = False

                if input_acao == "S":
                    sair = True

            if acao_rua:
                clear_output(wait=True)

                opcoes = ["Adicionar valor", "Remover valor", "Limpar vetor", "Voltar"]
                input_rua = seleciona_escolha(f"{cultura_atual.nome} > Ações > Ruas", opcoes)

                if input_rua == 1:
                    valor_rua = input_numerico("\n>> Digite o valor em metros da rua : ")
                    cultura_atual.add_rua(rua=valor_rua)

                if input_rua == 2:
                    cultura_atual.remover_rua()

                if input_rua == 3:
                    cultura_atual.limpa_ruas()

                if input_rua == 4:
                    acao_rua = False

                if input_rua == "S":
                    sair = True

            if acao_insumo:
                clear_output(wait=True)

                opcoes = ["Definir produto", "Remover produto", "Voltar"]
                input_insumo = seleciona_escolha(f"{cultura_atual.nome} > Ações > Insumos", opcoes)

                if input_insumo == 1:
                    nome_produto = input("\n>> Digite o nome do produto: ")
                    valor_insumo = input_numerico("\n>> Digite o novo valor do insumo(ML) por metro: ")
                    cultura_atual.adicionar_produto(nome_produto=nome_produto, insumo_por_metro=int(valor_insumo))

                if input_insumo == 2:
                    for produto, insumo in cultura_atual.produtos.items():
                        print(f"   - {produto}: {insumo} mL")

                    nome_produto = input("\n>> Digite o nome do produto a ser removido: ")
                    cultura_atual.remover_produto(nome_produto)

                if input_insumo == 3:
                    acao_insumo = False

                if input_insumo == "S":
                    sair = True

            if acao_calculo:
                clear_output(wait=True)

                opcoes = ["Área do plantio", "Manejo de insumos", "Histórico", "Voltar"]
                input_calculo = seleciona_escolha(f"{cultura_atual.nome} > Ações > Cálculos", opcoes)

                if input_calculo == 1:
                    opcoes = [str(c).capitalize() for c in cultura_atual.formas_geometricas]
                    indice = seleciona_escolha("Selecionar forma geométrica", opcoes)

                    forma_geo = cultura_atual.formas_geometricas[indice - 1]
                    calculo = cultura_atual.calcular_area_plantio(forma_geometrica=forma_geo)

                    input(">> Pressione uma tecla para continuar...")

                if input_calculo == 2:
                    if cultura_atual.ruas and cultura_atual.produtos:
                        total_insumos = cultura_atual.calcular_manejo_insumos()
                        for produto, quantidade in total_insumos.items():
                            print(f"Total de {produto} necessário: {quantidade} litros")
                        input(">> Pressione uma tecla para continuar...")
                    else:
                        print(">> Certifique-se de que as ruas e produtos estão definidos.")

                if input_calculo == 3:
                    cultura_atual.exibir_areas_calculadas()
                    input(">> Pressione uma tecla para continuar...")

                if input_calculo == 4:
                    acao_calculo = False

                if input_calculo == "S":
                    sair = True
            
            if acao_dados_meterologicos:
                clear_output(wait=True)
                utils = importr('utils')

                import sys
                import io
                import rpy2.rinterface_lib.callbacks

                # Criar um fluxo vazio
                empty_stream = io.StringIO()

                # Substituir os callbacks padrões para redirecionar a saída
                rpy2.rinterface_lib.callbacks.consolewrite_print = lambda x: empty_stream.write(x)
                rpy2.rinterface_lib.callbacks.consolewrite_warnerror = lambda x: empty_stream.write(x)

                # Caminho para o script R
                arquivo_r = "dados_meterologicos.r"

                # Ler o script R
                # with open(arquivo_r, "r", encoding='utf-8') as file:
                #     script_r = file.read()
                with open(arquivo_r, "r", encoding='utf-8-sig') as file:  # Ou tente 'latin-1' se for o caso
                    script_r = file.read()

                # Executar o script R
                r(script_r)

                # Chamar a função acao_dados_metereologicos do R e capturar o retorno
                acao_dados_metereologicos_func = r['acao_dados_metereologicos']

                # Executa a função e captura os resultados como um dicionário
                output = utils.capture_output(acao_dados_metereologicos_func())
                
                resultados = acao_dados_metereologicos_func()

                avg_precipitacao = resultados.rx2('avg_precipitacao')[0]  # Probabilidade média de precipitação
                avg_temperatura = resultados.rx2('avg_temperatura')[0]    # Temperatura média
                avg_vento = resultados.rx2('avg_vento')[0]                # Velocidade média do vento
                avg_umidade = resultados.rx2('avg_umidade')[0]            # Umidade média

                opcoes = [ 
                    "Probabilidade média de chuva", 
                    "Temperatura média", 
                    "Velocidade média de vento", 
                    "Volume médio de umidade",
                    "Voltar",
                ]  
                sair_menu_api = False
                # print(">> Selecione uma das opções para exibir a sua previsão média para os próximos dias:")
                while sair_menu_api == False:
                    input_numerico = seleciona_escolha("Exibir previsão",opcoes)

                    if input_numerico == 1:
                        print('teste')
                        print(f"A previsao da probabilidade media de chuva é:{avg_precipitacao}%")
                    elif input_numerico == 2:
                        print(f'A previsão de temperatura media para os proximos dias é:{avg_temperatura}°C')
                    elif input_numerico == 3:
                        print(f'A previsao da velocidade media do vento para os proximos dias é:{avg_vento}m/s')
                    elif input_numerico == 4:
                        print(f'A previsao  de umidade media para os proximos dias é:{avg_umidade}%')
                    elif input_numerico == 5:
                        sair_menu_api = True
                        acao_dados_meterologicos = False
                    elif input_calculo == "S":
                        sair = True
                    clear_output(wait=True)
            # Caminho para o script R

                # print("dados meterologicos teste")
                # from rpy2.robjects.packages import importr
                # # Captura a saída do R
                # 
            #     # Ativar a conversão automática entre pandas e data.frames do R
            #     pandas2ri.activate()
         
if __name__ == '__main__':
    main()


╔══════════════════════════════════╗
║         Exibir previsão          ║
╚══════════════════════════════════╝
║ 1 - Probabilidade média de chuva ║
║ 2 - Temperatura média            ║
║ 3 - Velocidade média de vento    ║
║ 4 - Volume médio de umidade      ║
║ 5 - Voltar                       ║
╚══════════════════════════════════╝
║ S - Sair                         ║
╚══════════════════════════════════╝


KeyboardInterrupt: Interrupted by user